In [2]:
#!pip install langchain transformers
!pip install langchain --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.0.10
    Uninstalling langchain-core-0.0.10:
      Successfully uninstalled langchain-core-0.0.10
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.346
    Uninstalling langchain-0.0.346:
      Successfully uninstalled langchain-0.0.346


In [1]:
from transformers import pipeline
from langchain import Chain, links

/Users/angieariza/Desktop/JAQLN-AI/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: cannot import name 'Chain' from 'langchain' (/Users/angieariza/Desktop/JAQLN-AI/myenv/lib/python3.9/site-packages/langchain/__init__.py)

In [ ]:

# Load the traffic violation dataset
data_path = 'path/to/your/traffic/violation/dataset.csv'
df = Chain.read_csv(data_path)

# Preprocess the data
df_cleaned = (
    df
    .dropna(subset=['Description', 'Charge'])  # Assuming 'Description' and 'Charge' columns are relevant
    .rename(columns={'Description': 'text', 'Charge': 'label'})
    .select(['text', 'label'])
)

# Create a Langchain QA chain for training
qa_chain = (
    Chain
    .link(links.split_train_test(test_size=0.2, random_state=42))
    .chain('train', lambda x: x[0])
    .chain('test', lambda x: x[1])
)

# Train a simple QA model using the transformers library
qa_model = pipeline('question-answering')
qa_chain.train(
    model=qa_model,
    input_col='train',
    output_col='predictions',
    target_col='test',
    query_col='text',
    target_query_col='text',
    query_suffix='_QA',
    target_suffix='_QA'
)

# Create a Conversational Retrieval QA chain
conv_retrieval_qa_chain = (
    Chain
    .link(links.merge(['text', 'label'], suffix='_QA'))
    .chain('retrieve', lambda x: x[['text_QA', 'label_QA']])
)

# Evaluate the model on the test set
conv_retrieval_qa_chain.evaluate(
    model=qa_model,
    input_col='test',
    output_col='predictions',
    query_col='text_QA',
    target_col='label_QA'
)
